In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

In [2]:
train_data = data.filter(pl.col("Group") != "5")
test_data = data.filter(pl.col("Group") == "5")

In [3]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [4]:
# fits model
model = rs.glm(
    formula="ClaimCount ~ TE(VehBrand) + bs(BonusMalus, df=4) + bs(VehAge, df=4) + bs(DrivAge, df=4) + C(Area) + bs(VehPower, df=4) + TE(VehGas) + C(Area):bs(VehAge, df=4) + C(Area):bs(DrivAge, df=4)",
    data=train_data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [5]:
print(model.summary())

                                 GLM Results                                  

Family:              NegativeBinomial(theta=1.9398) No. Observations:        542055
Link Function:       (default)       Df Residuals:            542005
Method:              IRLS + Ridge    Df Model:                    49
Scale:               0.2899          Alpha (λ):               0.0000
L1 Ratio:            0.00            Iterations:                   9
Non-zero coefs:      49             

Log-Likelihood:         -112381.1201 Deviance:                157137.7246
AIC:                     224862.2402 Null Deviance:            58319.4657
BIC:                     225422.3964 Pearson chi2:             1414368.40
Converged:           True           

------------------------------------------------------------------------------
Variable                         Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept

In [6]:
#writes analysis/diagnostics
model.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ TE(VehBrand) + bs(BonusMalus, df=4) + bs(VehAge, df=4) + bs(DrivAge, df=4) + C(Area) + bs(VehPower, df=4) + TE(VehGas) + C(Area):bs(VehAge, df=4) + C(Area):bs(DrivAge, df=4)', 'family': 'NegativeBinomial(theta=1.9398)', 'link': 'log', 'n_observations': 542055, 'n_parameters': 50, 'degrees_of_freedom_residual': 542005, 'converged': True, 'iterations': 9}, convergence_details=ConvergenceDetails(max_iterations_allowed=25, iterations_used=9, converged=True, reason='converged'), fit_statistics={'deviance': 157137.72460061286, 'null_deviance': 58319.465749978524, 'deviance_explained': -1.6944301114533222, 'log_likelihood': -112381.12011418647, 'aic': 224862.24022837295, 'bic': 225422.39636593737, 'pearson_chi2': 1414368.39940116, 'dispersion_deviance': 0.2899193265756088, 'dispersion_pearson': 2.6095117192667225}, loss_metrics={'family_deviance_loss': 0.2898924857512696, 'mse': 0.05417615843116408, 'mae': 0.09757409995663102, 'rmse': 0